### Dynamic Fee: Getting Data with Spark
Se bajan las acciones de los visitantes del sitio en las últimas 4 semanas para un site para el producto hoteles. Nos quedamos con los flows search, detail, checkout y thanks. 

El script para bajar los eventos esta separado para poder hacer el resto del análisis a la vez, si se tiene otro set de datos.

In [ ]:
import datetime
import wasabisql
import pytz
import pandas as pd
from pyspark.sql.functions import udf, lit
from math import radians, cos, sin, asin, sqrt, atan2
import numpy as np
from functools import reduce
from pyspark.sql import DataFrame

In [ ]:
sc.applicationId

### Importo 4 semanas de datos
- En start y end ponemos la fecha de inicio y fin del periodo del que queremos bajar los eventos. 
- generateDays genera una serie de sub starts y ends de un dia de duracion entre esas fechas.
- getEvents baja un dataframe por cada día de eventos y lo filtra.
- Cada dataframe se appendea directamente a un archivo parquet que se va sobreescribiendo, sin necesidad de hacer un checkpoint o de ir acumulando los dataframes en una lista.
 
EL proceso se hace de a un día porque a Spark le cuesta trabajar con tantos archivos avro en los que está distribuida la data.

In [ ]:
loader = wasabisql.SOTLoader(sqlContext)

In [ ]:
campos = ['userid', 'datetime', 'pr', 'fl', 'cc', 'ci', 'co', 'dc', 'hr', 'hid', 'di','hc', 'pri', 
                               'pritax', 'cur', 'exch', 'xClient', 'dtype', 'dbr']  
campos_sin_pr = ['userid', 'datetime','fl', 'cc', 'ci', 'co', 'dc', 'hr', 'hid', 'di','hc', 'pri', 
        
                 'pritax', 'cur', 'exch', 'xClient', 'dtype', 'dbr']

site = "CL"

In [ ]:
def generateDays(start, end):
    one_day = datetime.timedelta(1)
    i = 0
    while start + one_day*i < end:
        yield start + one_day*i
        i+=1

In [ ]:
def getEventsDay(start, end):

    events = loader.getEvents(campos, start, end)

    events = events.filter("length(userid) = 36") \\
        .filter('lower(fl) in ("search", "detail", "checkout", "thanks")') \\                  
        .filter('lower(pr) in ("hotels")') \\
        .filter("dc is not null") \\       
        .filter('lower(cc) not in ("cl", "ar", "br", "mx")')
                    
    #saco la columna de pr, ya que solo me quede con hoteles. Sacaria cc pero la uso mas adelante
    events = events[campos_sin_pr]
    
    # Ahora el checkpoint se hace con cada events diario
    #events = checkpoint(events)
    #events.count()
    return events

In [ ]:
mode = 'overwrite'
for start in generateDays(datetime.datetime(2016,6,28), datetime.datetime(2016,7,25)):
    end = start + datetime.timedelta(1)
    events = getEventsDay(start, end)
    events.write.parquet('dataset/DynFeeEvents-OTHERS-2016-06-27-2016-07-25.parquet', mode=mode)
    mode = 'append'

#### Uno todas las semanas

In [ ]:
# allevents es una lista de DataFrames
events = reduce(DataFrame.unionAll, [allevents])
# events es un DataFrame

In [ ]:
events.write.option("header", "true").save('dataset/DynFeeEvents-BR-2016-07-06-2016-07-17.parquet', format="parquet", mode="Overwrite")